## Библиотеки 

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## Функции

In [2]:
def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

## Получение данных

In [3]:
df_train = pd.read_csv('df_train.csv', sep=';')
df_test = pd.read_csv('df_test.csv', sep=';')

In [4]:
df_train['Data'] = df_train.Data.apply(lambda s: list(map(int, s.split(','))))
df_train['Target'] = df_train.Target.apply(lambda s: list(map(int, s.split(','))))
df_test['Data'] = df_test.Data.apply(lambda s: list(map(int, s.split(','))))

## Ранжирование по вероятностям
В модели для ранжирования кодов я учитываю временной контекст, то есть более новым транзакциям в последовательности даю больший вес за счет учета их позиций. К примеру, для входной последовательности `[1, 1, 1, 1, 1, 2, 2, 2]` выход будет `[2, 1]`, поскольку сумма позиций для двойки `(6+7+8 = 21)` будет больше, чем для единицы `(1+2+3+4+5 = 15)`

In [5]:
total_top_10 = (df_train['Data'] + df_train['Target'] + df_test['Data']).explode().value_counts(ascending=False).index[:10]
alpha = 1.7
max_length = int(np.mean([len(x) for x in df_test['Data']]) / alpha)

In [9]:
max_length

280

In [6]:
def tops_by_weight(seq, max_length=max_length, drop_from=2):
    seq = seq[-max_length:]
    seq_len = len(seq)
    pos_sum = seq_len * (seq_len+1) / 2
    codes = {}

    for code in np.unique(seq):
        positions = np.where(np.array(seq) == code)[0] + 1
        if len(positions) >= drop_from:
            codes[code] = sum(positions) / pos_sum
    
    output = sorted(codes, key=codes.get, reverse=True)
    
    if len(output) < 10:
        output += [x for x in total_top_10 if x not in output]

    return output[:10]

In [7]:
df_train['tops_by_weight'] = df_train['Data'].apply(tops_by_weight, drop_from=2)
print(mapk(df_train['Target'], df_train['tops_by_weight']))

0.33647650306153076


In [8]:
# df_test['Predicted'] = df_test['Data'].apply(tops_by_weight, drop_from=2)

# submission = df_test[['Id', 'Predicted']]
# submission['Predicted'] = submission['Predicted'].apply(lambda x: str(x).replace(',', '')[1:-1])
# submission.to_csv('submission_tops_by_proba_final_2.csv', index=False)